In [6]:
import tensorflow as tf
import torch
import torch.nn.functional as F

import numpy as np

print(tf.__version__)
print(torch.__version__)
print(np.__version__)

2.4.1
1.9.0+cpu
1.20.1


## Compare `binary_cross_entropy_with_logits`

In [19]:
y_true = np.random.uniform(size=[2, 128, 128])
y_pred = np.random.uniform(size=[2, 128, 128])

print(y_pred.shape)
print(y_true.shape)

(2, 128, 128)
(2, 128, 128)


In [41]:
tf_loss = tf.keras.losses.binary_crossentropy(
    y_true=y_true, y_pred=y_pred, from_logits=True,
)

print(tf_loss.shape)
print(tf.reduce_mean(tf_loss, axis=1))
print(sum(tf.reduce_mean(tf_loss, axis=1)))
print(tf.reduce_sum(tf.reduce_mean(tf_loss, axis=1)))

(2, 128)
tf.Tensor([0.73223665 0.73234737], shape=(2,), dtype=float64)
tf.Tensor(1.464584024483333, shape=(), dtype=float64)
tf.Tensor(1.464584024483333, shape=(), dtype=float64)


In [39]:
torch_loss = F.binary_cross_entropy_with_logits(
    torch.from_numpy(y_pred), torch.from_numpy(y_true), reduction="none"
)

print(torch_loss.shape)
print(torch_loss.mean(2).shape)
print(torch_loss.mean(2).mean(1))
print(sum(torch_loss.mean(2).mean(1)))

torch.Size([2, 128, 128])
torch.Size([2, 128])
tensor([0.7322, 0.7323], dtype=torch.float64)
tensor(1.4646, dtype=torch.float64)


## Compare `sigmoid_l1_loss`

In [33]:
logits = np.random.uniform(size=[2, 128, 128])
target = np.random.uniform(size=[2, 128, 128])
offset = -0.5
mask = np.random.uniform(size=[2, 128, 128])

In [52]:
def sigmoid_l1_loss_torch(logits, target, offset=0.0, mask=None):
    logp = torch.sigmoid(logits) + offset
    loss = torch.abs(logp - target)
    if mask is not None:
        w = mask.mean(2, True).mean(1, True)
        w[w == 0] = 1
        loss = loss * (mask / w)

    loss = loss.mean(2).mean(1)
    print(loss)
    return loss

sigmoid_l1_loss_torch(
    torch.from_numpy(logits),
    torch.from_numpy(target), offset, torch.from_numpy(mask))

tensor([0.3996, 0.3989], dtype=torch.float64)


tensor([0.3996, 0.3989], dtype=torch.float64)

In [53]:
def sigmoid_l1_loss_tf(logits, target, offset=0.0, mask=None):
    logp = tf.math.sigmoid(logits) + offset
    loss = tf.math.abs(logp - target)
    if mask is not None:
        w = tf.math.reduce_mean(
            tf.math.reduce_mean(mask, axis=2, keepdims=True),
            axis=1, keepdims=True
        )
        condition = tf.equal(w, 0)
        w = tf.where(condition, 1.0, w)
        loss = loss * (mask / w)

    loss = tf.math.reduce_mean(
        tf.math.reduce_mean(loss, axis=2),
        axis=1
    )
    return loss

tf_sig_l1_loss = sigmoid_l1_loss_tf(logits, target, offset, mask)
print(tf_sig_l1_loss)

tf.Tensor([0.39962673 0.39888739], shape=(2,), dtype=float64)


In [56]:
n_jtyp = 1
joff = np.random.uniform(size=[1, 2, 2, 128, 128])
t_joff = np.random.uniform(size=[1, 2, 2, 128, 128])
t_jmap = np.random.uniform(size=[1, 2, 128, 128])

In [57]:
torch_loss = sum(
    sigmoid_l1_loss_torch(
        torch.from_numpy(joff[i, j]), torch.from_numpy(t_joff[i, j]),
        -0.5, torch.from_numpy(t_jmap[i])
    )
    for i in range(n_jtyp)
    for j in range(2)
)
print(torch_loss)

tensor([0.4003, 0.3977], dtype=torch.float64)
tensor([0.3984, 0.4003], dtype=torch.float64)
tensor([0.7986, 0.7980], dtype=torch.float64)


In [59]:
tf_loss = sum(
    sigmoid_l1_loss_tf(
        joff[i, j], t_joff[i, j],
        -0.5, t_jmap[i]
    )
    for i in range(n_jtyp)
    for j in range(2)
)
print(tf_loss)

tf.Tensor([0.79864378 0.79797389], shape=(2,), dtype=float64)
